In [1]:
import torch
from torch import nn
from torch.nn import functional as F

from torch.nn import Conv2d

In [8]:
enc_features2 = torch.randn(1, 32, 256, 256)

dec_features2 = torch.randn(1, 32, 256, 256) 

In [26]:
test_data = torch.randn(1, 32, 1, 1)
C1 = 32
N= int(1/16 * C1)
test_data = test_data.squeeze(3)# test_data.view(1, C1, 1)

conv1x1_enc = nn.Conv1d(
            in_channels=C1,  
            out_channels=N, 
            kernel_size=1, 
            stride=1, 
        )

out = conv1x1_enc(test_data)

In [27]:
test_data.shape

torch.Size([1, 32, 1])

In [28]:
# [1, N, 1,1]
out1 = out.unsqueeze(3) #out.view(1, N, 1, 1)
out.shape, out1.shape

(torch.Size([1, 2, 1]), torch.Size([1, 2, 1, 1]))

In [6]:
class Spatial_Attention(nn.Module):
    def __init__(self,  enc_in_channels, dec_in_channels):
        super(Spatial_Attention, self).__init__()
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=1, padding=0)
        self.avg_pool = nn.AvgPool2d(kernel_size=2, stride=1, padding=0)
        self.conv1x1_enc = nn.Conv1d(
            in_channels=64,  
            out_channels=1, 
            kernel_size=3, 
            stride=2, 
            padding=1
        )
        
        
        
        self.conv1x1_dec = nn.Conv2d(
            in_channels=64,
            out_channels=1,  
            kernel_size=3,
            stride=2,
            padding=1
        )
        
        
        # self.final_conv = nn.Conv2d(
        #     in_channels=2,  
        #     out_channels=enc_in_channels, 
        #     kernel_size=7,  
        #     padding=3
        # )
        
        self.final_conv = nn.Conv2d(
            in_channels=1,  
            out_channels=enc_in_channels, 
            kernel_size=3,  
            padding=1
        )
        
        
    def forward(self, enc_features, dec_features):
        # average pooling
        enc_avg_out = self.avg_pool(enc_features)
        enc_max_out = self.max_pool(enc_features)
        enc_1x1_conv = self.conv1x1_enc(enc_avg_out)
        enc_1x1_co = self.conv1x1_enc(enc_max_out)
        addition_enc = enc_1x1_conv + enc_1x1_co
        
        dec_avg_out = self.avg_pool(dec_features)
        dec_max_out = self.max_pool(dec_features)
        dec_1x1_conv = self.conv1x1_dec(dec_avg_out)
        dec_1x1_co = self.conv1x1_dec(dec_max_out)
        addition_dec = dec_1x1_conv + dec_1x1_co
        
        addition_enc_dec = addition_enc + addition_dec
        
        final_1x1_conv = self.final_conv(addition_enc_dec)
        
        final_output = torch.sigmoid(final_1x1_conv)
        
        
        return final_output
        
        
        
        
        
        
        
        

In [7]:
model = Spatial_Attention(enc_in_channels=2, dec_in_channels=2)
enc_features = torch.randn(1, 64, 64, 64)  # Encoder features
dec_features = torch.randn(1, 64, 64, 64)  # Decoder features

output = model(enc_features, dec_features)
print(output.shape)  

torch.Size([1, 2, 32, 32])


In [39]:
# model = Spatial_Attention(enc_in_channels=2, dec_in_channels=2)
# enc_dec_features = model(enc_features2, dec_features2)
# enc_dec_features.shape

torch.Size([1, 2, 255, 255])